# Introduction to ExoCTK
Authors: Jules Fowler, Joe Filippazzo, Matthew Bourque

The Exoplanet Characterization Tool Kit (ExoCTK) is an open source Python 3.5+ package that serves as a one-stop-shop for observation planning, data reduction, and scientific analysis of transiting exoplanet systems.

In this notebook, we will cover the installation of the software as well as an end-to-end example that demonstrates the core functionalities of each tool provided by the ExoCTK package.

# Installation
You must first have a working installation of ``anaconda`` or ``miniconda`` for Python 3.  If you do not yet have this on your system, you can visit the following links for download and installation instructions:

- [Anaconda](https://www.anaconda.com/download/)
- [Miniconda](https://conda.io/en/latest/miniconda.html)

To obtain the `exoctk` package with the necessary environment files, you can either install the package via ``pip``

```
pip install exoctk
```

or, clone the repository directly from GitHub:

```
git clone https://github.com/ExoCTK/exoctk.git
cd exoctk
python setup.py [install|devlop]
```

To obtain the `exoctk` data, visit the [ExoCTK website](https://exoctk.stsci.edu/) and navigate to the "ExoCTK Data Download" tab. This will start a download of a zipped package of all the data you need to run
`exoctk`. The data will unzip in the appropriate structure, with the top layer
`exoctk_data`. Export an environment variable for `EXOCTK_DATA`. 

For Mac OS/Linux, add the line

```
export EXOCTK_DATA='/path/to/your/unzipped/directory/exoctk_data/'
```

to your `.bashrc` or `.bash_profile`. For Windows, add an evironment variable using System Utility. 


You can install the ExoCTK `conda` environment via the `env/environment-<PYTHON_VERSION>.yml` files (relative to the parent directory of where the repository was installed).  Note that there are separate environment files for each version of `python` that `exoctk` supports.  Ensure that `conda` is up to date, create the `exoctk` environment with the appropriate Python version, and activate it with:

```
conda update conda
conda activate base
conda env create -f environment-<PYTHON_VERSION>.yml
conda activate exoctk-<PYTHON_VERSION>
```

Finally, install `exoctk` with:

```
pip install exoctk
```

or by running the `exoctk` setup script:

```
python setup.py [install|develop]
```

# Imports
Here are a few basic imports we'll need later on...

In [38]:
import os
import numpy as np
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Transiting exoplanet system example
For this end-to-end demonstration, we will use the transiting hot Jupiter WASP-79b. Let's grab all the relevant system parameters using the [ExoMAST](https://exo.mast.stsci.edu) service lookup in ExoCTK.

In [11]:
from exoctk.utils import get_target_data
wasp79, url = get_target_data('WASP-79b')

# Generate Limb Darkening Coefficients
To generate limb darkening coefficients for our host star, we can use the `exoctk.limb_darkening` tool. We can create an `LDC` object using a stellar intensity grid such as the Kurucz ATLAS9 models (provided in the `EXOCTK_DATA` zip file).

In [27]:
from exoctk.limb_darkening.limb_darkening_fit import LDC
from exoctk.modelgrid import ModelGrid
from exoctk.utils import MODELGRID_DIR

atlas9 = ModelGrid(os.path.join(MODELGRID_DIR, 'ATLAS9'), resolution=700)
ld = LDC(atlas9)

330 models loaded from /Users/jfilippazzo/Documents/STScI/ExoCTK/EXOCTK_DATA/modelgrid/ATLAS9/


Next, we want to generate our limb darkening coefficients using a particular grism throughput using the `svo_filters` package. For this example, we'll use the GR700XD grism to simulate some NIRISS SOSS mode observations of our target, split into 15 equal wavelength bins.

In [28]:
from svo_filters import Filter
gr700xd = Filter('NIRISS.GR700XD.1', n_bins=15)
gr700xd.plot()

Bandpass trimmed to 0.54982 um - 2.799 um
15 bins of 149 pixels each.


Finally, we can calculate our limb darkening coefficients by passing the host star effective temperature, surface gravity, and metalicity to the `calculate` method. We will also provide the limb darkening profile name and the bandpass we wish to consider:

In [37]:
teff = wasp79['Teff']
logg = wasp79['stellar_gravity']
feh = wasp79['Fe/H']
ld.calculate(teff, logg, feh, 'quadratic', bandpass=gr700xd)
print(ld.results)

Interpolating grid point [6600.0/4.2/0.03]...


/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/table.py:791: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  newcol = col[slice_]


Run time in seconds:  2.183288097381592


UnitConversionError: Can only apply 'less' function to quantities with compatible dimensions

# Lightcurve Fitting
Let's say we extracted the 200 frames of time-series 1D spectra for our target and it looks like this:

In [45]:
tso_wave = np.linspace(gr700xd.wave_min, gr700xd.wave_max, 2048)
tso_flux = np.ones((200, 2048))
tso_time = np.arange(200)

We can fit a transit model to the data with the `lightcurve_fitting.LightCurve` class.

In [46]:
from exoctk.lightcurve_fitting.lightcurve import LightCurve
lc = LightCurve(tso_time, tso_flux)

TypeError: object of type 'NoneType' has no len()